In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
#alt.data_transformers.disable_max_rows()
# 替换 YOUR_API_KEY 为您从 Alpha Vantage 获取的API密钥
API_KEY = 'UTX653YK0YHRUAPO'
URL = "https://www.alphavantage.co/query"

# 设置请求参数
params = {
    "function": "FX_DAILY",
    "from_symbol": "USD",
    "to_symbol": "JPY",
    "apikey": API_KEY,
    "outputsize": "full",  # 或者使用 "compact" 仅获取最近100条数据
    "datatype": "json"
}

# 发送请求
response = requests.get(URL, params=params)
data = response.json()

In [4]:
# 解析数据
import altair as alt
df1 = pd.DataFrame(data['Time Series FX (Daily)']).T
df1.columns = ['open', 'high', 'low', 'close']  # 根据API的实际输出调整列名
df1 = df1.apply(pd.to_numeric)
df1['date'] = pd.to_datetime(df1.index)
df1.sort_values('date', inplace=True)
df1['Date'] = df1['date'].dt.strftime('%Y-%m-%d')  # 转换为字符串格式
df1['Exchange Rate'] = df1['close']  # 重命名为'Exchange Rate'以便在图表中使用
min_exchange_rate = df1['Exchange Rate'].min()

# 创建Altair图表
chart = alt.Chart(df1).mark_line().encode(
    x=alt.X('Date:T', axis=alt.Axis(title='Date', format='%Y-%m-%d'), timeUnit='yearmonthdate'),  # 使用timeUnit确保日期以年月日的形式显示
    y=alt.Y('Exchange Rate:Q', axis=alt.Axis(title='Exchange Rate'), scale=alt.Scale(domainMin=min_exchange_rate)),  # 设置最小值为数据的最小值
    color=alt.value('lightgrey'),
    tooltip=['Date:T', 'Exchange Rate:Q']  # 添加悬停提示工具
).properties(
    title='JPY/USD Exchange Rate Over Time',
    
    width=800  # 设置图表宽度
)
chart

alt.Chart(...)

In [7]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from pmdarima import auto_arima
import matplotlib.pyplot as plt


In [16]:
# 定义ARIMA模型
arima_model = auto_arima(df1['Exchange Rate'], seasonal=True, m=12,stepwise=True,
                         suppress_warnings=True, error_action="ignore", max_order=None, trace=True)

# 拟合模型
arima_model.fit(df1['Exchange Rate'])




Performing stepwise search to minimize aic
 ARIMA(2,1,2)(1,0,1)[12] intercept   : AIC=10152.345, Time=25.05 sec
 ARIMA(0,1,0)(0,0,0)[12] intercept   : AIC=10151.922, Time=1.18 sec
 ARIMA(1,1,0)(1,0,0)[12] intercept   : AIC=10148.951, Time=4.23 sec
 ARIMA(0,1,1)(0,0,1)[12] intercept   : AIC=10148.633, Time=4.73 sec
 ARIMA(0,1,0)(0,0,0)[12]             : AIC=10151.064, Time=0.61 sec
 ARIMA(0,1,1)(0,0,0)[12] intercept   : AIC=10150.379, Time=1.22 sec
 ARIMA(0,1,1)(1,0,1)[12] intercept   : AIC=10148.764, Time=13.19 sec
 ARIMA(0,1,1)(0,0,2)[12] intercept   : AIC=10148.112, Time=8.52 sec
 ARIMA(0,1,1)(1,0,2)[12] intercept   : AIC=10148.636, Time=31.15 sec
 ARIMA(0,1,0)(0,0,2)[12] intercept   : AIC=10149.759, Time=6.81 sec
 ARIMA(1,1,1)(0,0,2)[12] intercept   : AIC=10148.037, Time=12.73 sec
 ARIMA(1,1,1)(0,0,1)[12] intercept   : AIC=10148.438, Time=5.40 sec
 ARIMA(1,1,1)(1,0,2)[12] intercept   : AIC=10148.635, Time=39.04 sec
 ARIMA(1,1,1)(1,0,1)[12] intercept   : AIC=10148.651, Time=16.94 sec

In [22]:
forecast = arima_model.predict(n_periods=30)
print('Forecast:', forecast)

Forecast: 2024-04-26    155.631348
2024-04-29    155.625615
2024-04-30    155.632172
2024-05-01    155.657630
2024-05-02    155.670900
2024-05-03    155.655412
2024-05-06    155.665214
2024-05-07    155.663030
2024-05-08    155.679772
2024-05-09    155.671755
2024-05-10    155.680613
2024-05-13    155.693443
2024-05-14    155.665996
2024-05-15    155.661535
2024-05-16    155.660591
2024-05-17    155.638662
2024-05-20    155.629205
2024-05-21    155.636397
2024-05-22    155.629886
2024-05-23    155.630567
2024-05-24    155.624020
2024-05-27    155.625003
2024-05-28    155.614821
2024-05-29    155.606170
2024-05-30    155.606619
2024-05-31    155.606799
2024-06-03    155.606799
2024-06-04    155.606799
2024-06-05    155.606799
2024-06-06    155.606799
Freq: B, dtype: float64


In [24]:
# 创建时间序列的未来日期索引
import pandas as pd

# 假设 'forecast' 是你已经得到的预测数组
# 创建未来日期索引
future_dates = pd.date_range(start=df1['Date'].iloc[-1], periods=30, freq='D')

# 创建包含预测数据的DataFrame
forecast_df = pd.DataFrame({
    'Date': future_dates,
    'Exchange Rate': forecast,
    'Type': 'Forecast'  # 添加一个标记以区分预测和历史数据
})

# 转换日期格式以匹配原始数据
forecast_df['Date'] = forecast_df['Date'].dt.strftime('%Y-%m-%d')


In [30]:
df1 = df1.copy()
start_date = '2023-10-01'
df1['date'] = pd.to_datetime(df1['date'])
df1 = df1.loc[df1['date'] >= pd.to_datetime(start_date)]

In [31]:
import altair as alt
min_exchange_rate = df1['Exchange Rate'].min()
# 创建Altair图表
chart1 = alt.Chart(forecast_df).mark_line().encode(
    x=alt.X('Date:T', axis=alt.Axis(title='Date', format='%Y-%m-%d'), timeUnit='yearmonthdate'),
    y=alt.Y('Exchange Rate:Q', axis=alt.Axis(title='Exchange Rate'), scale=alt.Scale(domainMin=min_exchange_rate)),
    color=alt.Color('Type:N', legend=alt.Legend(title='Data Type')),  # 使用颜色来区分历史数据和预测数据
    tooltip=['Date:T', 'Exchange Rate:Q', 'Type:N']  # 添加悬停提示工具
).properties(
    title='JPY/USD Exchange Rate Over Time',
    width=800
)

chart3 = alt.Chart(forecast_df).mark_line().encode(
    x=alt.X('Date:T', axis=alt.Axis(title='Date', format='%Y-%m-%d'), timeUnit='yearmonthdate'),
    y=alt.Y('Exchange Rate:Q', axis=alt.Axis(title='Exchange Rate')),
    color=alt.Color('Type:N', legend=alt.Legend(title='Data Type')),  # 使用颜色来区分历史数据和预测数据
    tooltip=['Date:T', 'Exchange Rate:Q', 'Type:N']  # 添加悬停提示工具
).properties(
    title='JPY/USD Exchange Rate Over Time',
    width=800
)

chart2 = alt.Chart(df1).mark_line().encode(
    x=alt.X('Date:T', axis=alt.Axis(title='Date', format='%Y-%m-%d'), timeUnit='yearmonthdate'),
    y=alt.Y('Exchange Rate:Q', axis=alt.Axis(title='Exchange Rate')),
    color=alt.Color('Type:N', legend=alt.Legend(title='Data Type')),  # 使用颜色来区分历史数据和预测数据
    tooltip=['Date:T', 'Exchange Rate:Q', 'Type:N']  # 添加悬停提示工具
).properties(
    title='JPY/USD Exchange Rate Over Time',
    width=800
)

# 展示图表
chart1+chart2+chart3


alt.LayerChart(...)